# Datové zdroje

## Webové stránky

In [2]:
!pip install requests_ntlm

  Using cached requests_ntlm-1.1.0-py2.py3-none-any.whl (5.7 kB)
  Using cached ntlm_auth-1.5.0-py2.py3-none-any.whl (29 kB)


In [3]:
import getpass
import urllib.request, json
from requests_ntlm import HttpNtlmAuth

In [4]:
url = ''
people = [633]
password = getpass.getpass()
user = 'UNIVO\\stefeka'

 ··············


In [6]:
from functools import lru_cache
import requests

@lru_cache
def readTbl(UCO):
    url = f'https://apl.unob.cz/UOAPI/Rozvrhy/Osoba/{UCO}'
    page = requests.get(url, auth = HttpNtlmAuth(user, password))
    content = page.content.decode('utf8')
    return content

data = readTbl(633)
print(data[:1000])


   

<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>RozvrhyOsoba</title>
  <link href="/UOAPI/Content/css?v=oQnauHsqXdVBkuWhCiSRZpmRifmLkzjqiI7oah75sjQ1" rel="stylesheet"/>

  <script src="/UOAPI/bundles/modernizr?v=inCVuEFe6J4Q07A0AcRsbJic_UE5MwpRMNGcOtk94TE1"></script>

</head>
<body><table class="table table-striped table-condensed">
  <thead>
    <tr>
      <th>Datum</th>
      <th>Začátek</th>
      <th>Konec</th>
      <th>Typ lekce</th>
      <th>Předmět</th>
      <th>Předmět Uic</th>
      <th>Učebna</th>
      <th>Vyučující</th>
      <th>Vyučující Uco</th>
      <th>Učební skupina</th>
      <th>Učební skupina Uic</th>
    </tr>
  </thead>
  <tbody>
      <tr>
    <td>13. 10. 2020</td>
    <td>11:40</td>
    <td>13:10</td>
    <td>předn&#225;ška</td>
    
    

    


<td>Operačn&#237; anal&#253;za ve vojenstv&#237; / T1</td>
<td>01265</td


## Regulární výrazy

https://docs.python.org/3/library/re.html

In [19]:
import re

data = readTbl(633)
m = re.search(r'(?:<tr>)([^<]*(?:<td>[^<]*</td>[^<]*)*)(?:</tr>)', data)
radek = m.group(0)
print(radek)

<tr>
    <td>13. 10. 2020</td>
    <td>11:40</td>
    <td>13:10</td>
    <td>předn&#225;ška</td>
    
    

    


<td>Operačn&#237; anal&#253;za ve vojenstv&#237; / T1</td>
<td>01265</td>

    <td>DISTANČN&#205;</td>
    <td>Štefek A.</td>
    <td>633</td>
    <td>5ASV (2017)</td>
    <td>20953</td>
  </tr>


In [53]:
data = readTbl(633)
regTester = re.compile(r'(?:<tr>)([^<]*(?:<td>[^<]*</td>[^<]*)*)(?:</tr>)')
matches = regTester.findall(data)
result = []
for _ in matches:
    result.append(_)

for index, item in enumerate(result):
    print(index, item[:10])
    if index > 9:
        break

0 
    <td>
1 
    <td>
2 
    <td>
3 
    <td>
4 
    <td>
5 
    <td>
6 
    <td>
7 
    <td>
8 
    <td>
9 
    <td>
10 
    <td>


## Mapování

### Selector

In [15]:
def createSelector(keyName):
    def select(item):
        return item[keyName]
    return select

data = {'id': 633, 'page': readTbl(633)}
selector = createSelector('page')
print(f'got from item => ', selector(data)[:100])

got from item =>  
   

<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8" />
  <meta name="viewport" cont


### Mapper

In [35]:
import html

def getRows(item):
    regTester = re.compile(r'(?:<tr>)([^<]*(?:<td>[^<]*</td>[^<]*)*)(?:</tr>)')
    matches = regTester.findall(item)
    result = []
    for m in matches:
        result.append(html.unescape(m))
    return result

data = {'id': 633, 'page': readTbl(633)}
selector = createSelector('page')
rows = getRows(selector(data)) 
print(rows[0])
print('-'*30)
print(rows[1])


    <td>13. 10. 2020</td>
    <td>11:40</td>
    <td>13:10</td>
    <td>přednáška</td>
    
    

    


<td>Operační analýza ve vojenství / T1</td>
<td>01265</td>

    <td>DISTANČNÍ</td>
    <td>Štefek A.</td>
    <td>633</td>
    <td>5ASV (2017)</td>
    <td>20953</td>
  
------------------------------

    <td>14. 10. 2020</td>
    <td>11:40</td>
    <td>13:10</td>
    <td>přednáška</td>
    
    

    


<td>Operační analýza ve vojenství / T2</td>
<td>01265</td>

    <td>DISTANČNÍ</td>
    <td>Štefek A.</td>
    <td>633</td>
    <td>5ASV (2017)</td>
    <td>20953</td>
  


In [36]:
def getCells(item):
    regTester = re.compile(r'(?:<td>)([^<]*)(?:</td>)')
    matches = regTester.findall(item)
    result = []
    for _ in matches:
        result.append(_)
    return result

cells = getCells(rows[0])
print(cells)

['13. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T1', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']


### Capsule

In [57]:
def createCapsule(selector, func, coupling):
    def result(item):
        return coupling(item, func(selector(item)))
    return result

In [58]:
selector = selector = createSelector('id')
coupling = lambda item, result: {**item, 'page': result}
capsuleRead = createCapsule(selector, readTbl, coupling)

data = {'id': 633, 'page': readTbl(633)}
result = capsuleRead(data)
print(result.keys())
print(result['page'][:100])

dict_keys(['id', 'page'])

   

<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8" />
  <meta name="viewport" cont


In [59]:
selector = selector = createSelector('page')
coupling = lambda item, result: {**item, 'rows': result}
capsuleRows = createCapsule(selector, getRows, coupling)

data = {'id': 633, 'page': readTbl(633)}
result = capsuleRows(data)
print(result['id'])
print('-'*30)
print(result['page'][:100])
print('-'*30)
print(result['rows'][:2])

633
------------------------------

   

<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8" />
  <meta name="viewport" cont
------------------------------
['\r\n    <td>13. 10. 2020</td>\r\n    <td>11:40</td>\r\n    <td>13:10</td>\r\n    <td>přednáška</td>\r\n    \r\n    \r\n\r\n    \r\n\r\n\r\n<td>Operační analýza ve vojenství / T1</td>\r\n<td>01265</td>\r\n\r\n    <td>DISTANČNÍ</td>\r\n    <td>Štefek A.</td>\r\n    <td>633</td>\r\n    <td>5ASV (2017)</td>\r\n    <td>20953</td>\r\n  ', '\r\n    <td>14. 10. 2020</td>\r\n    <td>11:40</td>\r\n    <td>13:10</td>\r\n    <td>přednáška</td>\r\n    \r\n    \r\n\r\n    \r\n\r\n\r\n<td>Operační analýza ve vojenství / T2</td>\r\n<td>01265</td>\r\n\r\n    <td>DISTANČNÍ</td>\r\n    <td>Štefek A.</td>\r\n    <td>633</td>\r\n    <td>5ASV (2017)</td>\r\n    <td>20953</td>\r\n  ']


In [66]:
selectorR = createSelector('rows')
couplingR = lambda item, result: {**item, 'lessons': result}
processor = lambda array: list(map(getCells, array))

capsuleCells = createCapsule(selectorR, processor, couplingR)
data = {'id': 633, 'page': readTbl(633)}
result = capsuleRows(data)
result = capsuleCells(result)

print(result['lessons'][0])
print('-'*30)
print(result['lessons'][1])

['13. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T1', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']
------------------------------
['14. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T2', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']


## Redukce

In [65]:
from functools import reduce

def count(acc, item):
    return acc + 1

result = reduce(count, range(9), 0)
print(result)

9


In [75]:
capsuleCountLessons = createCapsule(
    createSelector('lessons'),
    len,
    lambda item, result: {**item, 'lessonsCount': result}
)

r2 = capsuleCountLessons(result)
print(r2['lessonsCount'])

14


## Multiple Functions

In [67]:
def joinFunc(*funcs):
    def process(data):
        result = data
        for func in funcs:
            result = func(result)
        return result
    return process

In [76]:
joinedFuncs = joinFunc(capsuleRead, capsuleRows, capsuleCells, capsuleCountLessons)
data = {'id': 633, 'page': readTbl(633)}
result = joinedFuncs(data)

print(result['lessons'][0])
print('-'*30)
print(result['lessons'][1])

['13. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T1', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']
------------------------------
['14. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T2', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']


## Multiple data

In [77]:
source = [633, 571]
dataM = [{'id': id} for id in source]
results = list(map(joinedFuncs, dataM))

for result in results:
    print(result['lessonsCount'])
    print(result['lessons'][0])
    print('-'*30)
    print(result['lessons'][1])    
    print('='*30)

101
['13. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T1', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']
------------------------------
['14. 10. 2020', '11:40', '13:10', 'přednáška', 'Operační analýza ve vojenství / T2', '01265', 'DISTANČNÍ', 'Štefek A.', '633', '5ASV (2017)', '20953']
14
['13. 10. 2020', '9:50', '11:20', 'přednáška', 'Konstrukce a provoz protiletadlových systémů / T5', '05348', 'DISTANČNÍ', 'Hamtil I.', '571', '5ASV (2017)', '20953']
------------------------------
['14. 10. 2020', '9:50', '11:20', 'přednáška', 'Konstrukce a provoz protiletadlových systémů / T6', '05348', 'DISTANČNÍ', 'Hamtil I.', '571', '5ASV (2017)', '20953']
